In [2]:
import pickle
import numpy as np
import pandas as pd

import os, sys, random, math, time, socket, struct

import scipy.signal

bufferpath = "../../python/signalProc"
sys.path.append(bufferpath)
import bufhelp

with open('Merian.pickle', 'rb') as file:
    data = pickle.load(file)

In [67]:
import matplotlib.pyplot as plt
% matplotlib inline
m = np.asmatrix(data['data'][20])
m = m[:, 0:9]

In [12]:
from scipy.fftpack import fft

def epoch2vector(epoch):
    X2 = np.matrix(epoch)
    X3 = scipy.signal.detrend(X2)
    X4 = np.dot(np.eye(X3.shape[0]) - (1.0 / X3.shape[0]), X3)
    X5 = fft(X4[:, 0:9], axis=0)
    
    # Add amplitudes to the final vector
    X6 = np.absolute(X5)
    X7 = X6[8:30]
    X8 = X7.reshape(-1, 1)
    
    # Add phases to the final vector
    X9 = np.angle(X5)
    X10 = X9.reshape(-1, 1)
    
    # Combine the vectors
    X11 = np.vstack([X8, X10]).reshape(-1, 1)
    
    #return X11
    return X8

In [21]:
for participant in ['Merian', 'Monica', 'Nathaly']:
    with open('%s.pickle' % participant, 'rb') as file:
        data = pickle.load(file)
        X = data['data']
        data = []
        for epoch in X:
            v = epoch2vector(epoch)
            data.append(np.squeeze(np.asarray(v)))
        X = np.asmatrix(data)
        with open('%s_data.pickle' % participant, 'wb') as output_file:
            pickle.dump(X, output_file)

In [28]:
X, y = None, None
for participant in ['Merian', 'Monica', 'Nathaly']:
    with open('%s_labels.pickle' % participant, 'rb') as label_file:
        labels = pickle.load(label_file)
        y_participant = np.asmatrix(labels['hot'])
    with open('%s_data.pickle' % participant, 'rb') as data_file:
        X_participant = pickle.load(data_file)
    X = X_participant if X is None else np.vstack([X, X_participant])
    y = y_participant if y is None else np.vstack([y, y_participant])
    
with open('combined.pickle', 'wb') as file:
    pickle.dump([X, y], file)

In [29]:
with open('combined.pickle', 'rb') as file:
    X, y = pickle.load(file)
print(X.shape, y.shape)

(1620, 198) (1620, 4)


In [38]:
from preproc import *

X = m

# Detrend
X = scipy.signal.detrend(X)

# CAR filter
X = np.dot(np.eye(X.shape[0]) - (1.0/X.shape[0]), X)

In [62]:
from scipy.fftpack import fft

data = fft(X[:, 0:9], axis=0)
data = np.absolute(data)
data = data[8:28]
A = data.reshape(-1, 1)
print(A.shape)
#print(amplitudes.shape)
#plt.plot(np.real(r))
#plt.plot(np.absolute(data))

(180, 1)
